# Songwriting and Language Generation using TensorFlow
## Writing a sonnet in the style of William Shakespeare, using a RNN

This project is based heavily upon the first lab exercise from MIT Deep Learning 6.S191, in which a Recurrent Neural Network (RNN) is used to generate synthetic music based upon Irish folk songs.

I have adapted the code to generate Shakespearean Sonnets rather than music, using the entirety of the sonnets of William Shakespeare to train the model.

For anyone else looking to learn more about Deep Learning I thoroughly recommend checking out the lectures and exercises from the aforementioned MIT open course at http://introtodeeplearning.com/

Sonnets sourced from http://www.shakespeares-sonnets.com/all.php

Source code used under the MIT License.
© MIT 6.S191: Introduction to Deep Learning

http://introtodeeplearning.com

In [6]:
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests

In [ ]:
# Check that we are using a GPU, if not switch runtimes
#   using Runtime > Change Runtime Type > GPU
assert len(tf.config.list_physical_devices('GPU')) > 0

In [21]:
url = 'http://www.shakespeares-sonnets.com/all.php'
page = requests.get(url)

# create an instance of the BeautifulSoup class, which will parse the html (content) from the requests response
soup = BeautifulSoup(page.content, 'html.parser')
alltext = soup.get_text()

In [22]:
# trim to retrieve only the sonnets from the webpage
alltext = alltext[alltext.index('All Sonnets'):alltext.index('Copyright')]
print(alltext)

All Sonnets
I.
From fairest creatures
we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou contracted to thine own bright eyes,
Feed'st thy light's flame with self-substantial fuel,
Making a famine where abundance lies,
Thy self thy foe, to thy sweet self too cruel:
Thou that art now the world's fresh ornament,
And only herald to the gaudy spring,
Within thine own bud buriest thy content,
And, tender churl, mak'st waste in niggarding:
Pity the world, or else this glutton be,
To eat the world's due, by the grave and thee.
II.
When
forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held: 
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days; 
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more

In [29]:
regex = "[XICVL]+\.\n"
alltext = re.sub(regex, "", alltext)
print(alltext)

All Sonnets
From fairest creatures
we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou contracted to thine own bright eyes,
Feed'st thy light's flame with self-substantial fuel,
Making a famine where abundance lies,
Thy self thy foe, to thy sweet self too cruel:
Thou that art now the world's fresh ornament,
And only herald to the gaudy spring,
Within thine own bud buriest thy content,
And, tender churl, mak'st waste in niggarding:
Pity the world, or else this glutton be,
To eat the world's due, by the grave and thee.
When
forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held: 
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days; 
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise

In [30]:
alltext = alltext.replace("\n"," ").lower()
print(alltext)

 within and suffer dearth, painting thy outward walls so costly gay? why so large cost, having so short a lease, dost thou upon thy fading mansion spend? shall worms, inheritors of this excess, eat up thy charge? is this thy body's end? then soul, live thou upon thy servant's loss, and let that pine to aggravate thy store; buy terms divine in selling hours of dross; within be fed, without be rich no more: so shall thou feed on death, that feeds on men, and death once dead, there's no more dying then. my love is as a fever longing still, for that which longer nurseth the disease; feeding on that which doth preserve the ill, the uncertain sickly appetite to please. my reason, the physician to my love,  angry that his prescriptions are not kept, hath left me, and i desperate now approve desire is death, which physic did except. past cure i am, now reason is past care, and frantic-mad with evermore unrest; my thoughts and my discourse as madmen's are, at random from the truth vainly expres

In [31]:
alltext = alltext.split(' ')
print(alltext)

['all', 'sonnets', 'from', 'fairest', 'creatures\r', 'we', 'desire', 'increase,', 'that', 'thereby', "beauty's", 'rose', 'might', 'never', 'die,', 'but', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease,', 'his', 'tender', 'heir', 'might', 'bear', 'his', 'memory:', 'but', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes,', "feed'st", 'thy', "light's", 'flame', 'with', 'self-substantial', 'fuel,', 'making', 'a', 'famine', 'where', 'abundance', 'lies,', 'thy', 'self', 'thy', 'foe,', 'to', 'thy', 'sweet', 'self', 'too', 'cruel:', 'thou', 'that', 'art', 'now', 'the', "world's", 'fresh', 'ornament,', 'and', 'only', 'herald', 'to', 'the', 'gaudy', 'spring,', 'within', 'thine', 'own', 'bud', 'buriest', 'thy', 'content,', 'and,', 'tender', 'churl,', "mak'st", 'waste', 'in', 'niggarding:', 'pity', 'the', 'world,', 'or', 'else', 'this', 'glutton', 'be,', 'to', 'eat', 'the', "world's", 'due,', 'by', 'the', 'grave', 'and', 'thee.', 'when\r', 'forty', 'winters', 'shall', 'besiege',

In [40]:
useless_characters = ["(", ")", "\"", "\'", "", ":", ";", ",", ".", "!", "?", "\“", "\…", "<u+203d>", "\r", "\xa0", "-"]
clean_text = list(set(alltext))
for character in useless_characters:
    clean_text = [word.replace(character,"") for word in clean_text]
print(clean_text)

['', 'oertake', 'was', 'cold', 'hes', 'sunk', 'shames', 'accident', 'prone', 'discloses', 'figured', 'swayst', 'same', 'knowledge', 'blame', 'too', 'world', 'confounding', 'remember', 'harvest', 'user', 'longer', 'fault', 'ride', 'neglect', 'kill', 'follow', 'accessary', 'deaf', 'hateth', 'conspire', 'divert', 'wealth', 'rarities', 'tresses', 'withering', 'staineth', 'deface', 'trophies', 'writ', 'reeks', 'slanderers', 'sets', 'fulness', 'league', 'make', 'face', 'save', 'purposed', 'assemble', 'tickled', 'buried', 'mute', 'fill', 'envy', 'frailties', 'quick', 'compass', 'oaths', 'tincture', 'breast', 'travail', 'beautiful', 'conquered', 'written', 'cheek', 'blenches', 'march', 'diest', 'motion', 'admitted', 'verse', 'tyranny', 'pale', 'live', 'see', 'unlearned', 'sullied', 'rehearse', 'willingly', 'spoils', 'besides', 'are', 'blamed', 'hath', 'fairer', 'random', 'thither', 'grace', 'charactered', 'loss', 'rearward', 'men', 'bow', 'wanting', 'metre', 'good', 'hand', 'methods', 'poesy',

In [41]:
# Find all unique characters in the joined string
vocab = sorted(set(clean_text))
print(vocab)
print("There are", len(vocab), "unique words in the sonnets")

['', 'a', 'abhor', 'abide', 'able', 'about', 'above', 'absence', 'absent', 'abundance', 'abundant', 'abuse', 'abused', 'abuses', 'abysm', 'accents', 'acceptable', 'acceptance', 'accessary', 'accident', 'accidents', 'account', 'accumulate', 'accuse', 'accusing', 'achieve', 'acknowledge', 'acquaintance', 'acquainted', 'act', 'action', 'active', 'actor', 'add', 'added', 'adders', 'addeth', 'adding', 'addition', 'adieu', 'adjunct', 'admire', 'admired', 'admiring', 'admit', 'admitted', 'adonis', 'adore', 'adoting', 'adulterate', 'advance', 'advantage', 'adverse', 'advisd', 'advocate', 'afar', 'affable', 'affairs', 'affections', 'afford', 'affords', 'afloat', 'afresh', 'after', 'afterloss', 'afterwards', 'again', 'against', 'age', 'ages', 'aggravate', 'ah', 'aid', 'air', 'alack', 'alas', 'alchemy', 'alien', 'alike', 'alive', 'all', 'allayed', 'alleating', 'allege', 'allow', 'alltheworld', 'almost', 'aloft', 'alone', 'already', 'alter', 'alteration', 'altered', 'altering', 'alters', 'although

In [ ]:
# Enumerate the vocab (assign each word a number) to create a mapping of words - numbers
word2idx = {u:i for i, u in enumerate(vocab)}

In [ ]:
# Reverse the mapping
idx2word = np.array(vocab)

In [ ]:
# Vectorize the lyrics
def vectorize_string(string):
  vectorized_words = []
  for word in string:
      vectorized_words.append(word2idx[word])
  vectorized_words = np.array(vectorized_words)
  return vectorized_words

print(clean_text)
vectorized_words = vectorize_string(clean_text)
print(vectorized_words)

In [ ]:
def get_batch(vectorized_words, seq_length, batch_size):
  # the length of the vectorized songs string
  n = vectorized_words.shape[0] - 1
  # randomly choose the starting indices for the examples in the training batch
  idx = np.random.choice(n-seq_length, batch_size)

  # construct a list of input sequences for the training batch
  input_batch = [vectorized_words[i:i+seq_length] for i in idx]
  # construct a list of output sequences for the training batch
  output_batch = [vectorized_words[i+1:i+1+seq_length] for i in idx]

  # x_batch, y_batch provide the true inputs and targets for network training
  x_batch = np.reshape(input_batch, [batch_size, seq_length])
  y_batch = np.reshape(output_batch, [batch_size, seq_length])

  return x_batch, y_batch

In [ ]:
# Demonstrate the batching over the timesteps
x_batch, y_batch = get_batch(vectorized_words, seq_length=5, batch_size=1)
print(x_batch)
print(y_batch)
for i, (input_idx, target_idx) in enumerate(zip(np.squeeze(x_batch), np.squeeze(y_batch))):
    print("Step {:3d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2word[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2word[target_idx])))

In [ ]:
def LSTM(rnn_units): 
  return tf.keras.layers.LSTM(
    rnn_units, 
    return_sequences=True, 
    recurrent_initializer='glorot_uniform',
    recurrent_activation='sigmoid',
    stateful=True,
  )

In [ ]:
### Defining the RNN Model ###

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    # Layer 1: Embedding layer to transform indices into dense vectors 
    #   of a fixed embedding size
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),

    # Layer 2: LSTM with `rnn_units` number of units. 
    LSTM(rnn_units),

    # Layer 3: Dense (fully-connected) layer that transforms the LSTM output
    #   into the vocabulary size. 
    tf.keras.layers.Dense(vocab_size)
  ])

  return model

# Build a simple model with default hyperparameters. You will get the 
#   chance to change these later.
model = build_model(len(vocab), embedding_dim=256, rnn_units=1024, batch_size=32)

In [ ]:
model.summary()

In [ ]:
x, y = get_batch(vectorized_words, seq_length=100, batch_size=32)
pred = model(x)
print("Input shape:      ", x.shape, " # (batch_size, sequence_length)")
print("Prediction shape: ", pred.shape, "# (batch_size, sequence_length, vocab_size)")

In [ ]:
# obtain predictions from untrained model
sampled_indices = tf.random.categorical(pred[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

In [ ]:
# Decode predictions from untrained model, find they're a bit rubbish
print(x)
print("Input: \n", repr(" ".join(idx2word[x[0]])))
print()
print("Next Word Predictions: \n", repr(" ".join(idx2word[sampled_indices])))

In [ ]:
### TRAINING THE MODEL: Part 1: Defining the loss function ###

# define the loss function to compute and return the loss between the true labels and predictions (logits). 
# Set the argument from_logits=True.
def compute_loss(labels, logits):
  loss = tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
  return loss

# compute the loss using the true next characters from the example batch 
# and the predictions from the untrained model several cells above
example_batch_loss = compute_loss(y, pred)

print("Prediction shape: ", pred.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

In [ ]:
### Hyperparameter setting and optimization ###

# Optimization parameters:
num_training_iterations = 2000  # Increase this to train longer
batch_size = 4  # Experiment between 1 and 64
seq_length = 100  # Experiment between 50 and 500
learning_rate = 5e-3  # Experiment between 1e-5 and 1e-1

# Model parameters: 
vocab_size = len(vocab)
embedding_dim = 256 
rnn_units = 1024  # Experiment between 1 and 2048

# Checkpoint location: 
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "my_ckpt")

In [ ]:
### Define optimizer and training operation ###

# instantiate a new model for training using the `build_model`
# function and the hyperparameters created above.'''
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)

# Instantiate an optimizer with its learning rate.
#   Checkout the tensorflow website for a list of supported optimizers.
#   https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/
#   Try using the Adam optimizer to start
optimizer = tf.keras.optimizers.Adam(learning_rate)


@tf.function
def train_step(x, y): 
  # Use tf.GradientTape()
  with tf.GradientTape() as tape:
  
    # Feed the current input into the model and generate predictions
    y_hat = model(x)
  
    # compute the loss
    loss = compute_loss(y, y_hat)

  # Now, compute the gradients 
#    complete the function call for gradient computation. 
#       Remember that we want the gradient of the loss with respect all 
#       of the model parameters. 
#       HINT: use `model.trainable_variables` to get a list of all model
#       parameters.
  grads = tape.gradient(loss, model.trainable_variables)
  
  # Apply the gradients to the optimizer so it can update the model accordingly
  optimizer.apply_gradients(zip(grads, model.trainable_variables))
  return loss

##################
# Begin training!#
##################

history = []
if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

pbar = tqdm(range(num_training_iterations))
for iter in pbar:

  # Grab a batch and propagate it through the network
  x_batch, y_batch = get_batch(vectorized_words, seq_length, batch_size)
  loss = train_step(x_batch, y_batch)

  # Update the progress bar
  history.append(loss.numpy().mean())
  pbar.set_description("loss: {}".format(loss.numpy().mean()))

  # Update the model with the changed weights!
  if iter % 100 == 0:     
    model.save_weights(checkpoint_prefix)
    
    
# Save the trained model and the weights
model.save_weights(checkpoint_prefix)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# Restore the model weights for the last checkpoint after training
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

model.summary()

In [ ]:
def generate_text(model, start_string, generation_length=1000):
  # Evaluation step (generating ABC text using the learned RNN model)

  # Convert the start string to numbers (vectorize)
  input_eval = vectorize_string(start_string)
  print(input_eval)
  input_eval = [word2idx[word] for word in start_string] 
  print(input_eval)
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Here batch size == 1
  model.reset_states()
  tqdm._instances.clear()

  for i in tqdm(range(generation_length)):
      # evaluate the inputs and generate the next character predictions
      predictions = model(input_eval)
      
      # Remove the batch dimension
      predictions = tf.squeeze(predictions, 0)
      
      # use a multinomial distribution to sample
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      
      # Pass the prediction along with the previous hidden state
      #   as the next inputs to the model
      input_eval = tf.expand_dims([predicted_id], 0)
      
      # add the predicted word to the generated text
      # Hint: consider what format the prediction is in vs. the output
      text_generated.append(idx2word[predicted_id])
    
  return ([start_string, text_generated])

In [ ]:
# Use the model and the function defined above to generate song lyrics of 100 words
# Choose a word that appears in the vocabulary (lower case) to seed the generator
generated_text = generate_text(model, start_string=["shake"], generation_length=100)

In [ ]:
print(generated_text)